git remote add origin https://github.com/HarshaM13/Assignment_14.git

git branch -M main

git push -u origin main

In [12]:
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib
import plotly
from dash import html, Dash
from dash import dcc
from dash.dependencies import Input, Output
import plotly.graph_objects as go

In [9]:
df = pd.read_csv("data/processed/df_covid_val_pred.csv")
df.head()

,Unnamed: 0,Date,Confirmed,Recovered,Deaths,Increase rate,year,month,Predicted_Deaths
0,0,2020-02-20,76216,18181,2248,0.741524,2020,Feb,300999.87
1,1,2020-02-21,76846,18894,2252,0.826598,2020,Feb,299955.43
2,2,2020-02-25,80412,27907,2710,1.073431,2020,Feb,296929.74
3,3,2020-03-28,670495,139421,32374,11.498481,2020,Mar,101595.88
4,4,2020-03-30,791975,164213,39895,8.978487,2020,Mar,131953.30


In [8]:

app = Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id="month",
        clearable=False,
        options=[{"label": m, "value": m} for m in df['month'].unique()],
        value=df['month'].unique()[0]
    ),
    dcc.Graph(id='graph')
])

@app.callback(
    Output('graph', 'figure'),
    Input('month', 'value')
)
def update_graph(selected_month):
    filtered_df = df[df['month'] == selected_month]
    fig = px.pie(filtered_df, names="Date", values="Deaths", title=f"Deaths in {selected_month}")
    return fig

if __name__ == "__main__":
    app.run(port=8060, debug=True) 


In [15]:
app = Dash(__name__)

app.layout = html.Div([
    html.H1("COVID Deaths Dashboard", style={"textAlign": "center"}),

    dcc.Dropdown(
        id="month",
        clearable=False,
        options=[{"label": m, "value": m} for m in df['month'].unique()],
        value=df['month'].unique()[0],
        style={"width": "50%", "margin": "auto"}
    ),

    html.Div([
        dcc.Graph(id='kpi_deaths'),  # ✅ Added KPI card
    ], style={"margin": "20px 0"}),

    html.Div([
        dcc.Graph(id='graph1'),
        dcc.Graph(id='graph2'),
        dcc.Graph(id='graph4')
    ], style={"display": "grid", "gridTemplateColumns": "1fr 1fr", "gap": "20px"})
])

@app.callback(
    [Output('graph1', 'figure'),
     Output('graph2', 'figure'),
     Output('kpi_deaths', 'figure'),
     Output('graph4', 'figure')],
    Input('month', 'value')
)
def update_graphs(selected_month):
    filtered_df = df[df['month'] == selected_month]

    # Graph 1: Bar chart of Deaths by Date
    fig1 = px.bar(filtered_df, x="Date", y="Deaths", title=f"Deaths in {selected_month}", text="Deaths")
    fig1.update_traces(textposition="outside")

    # Graph 2: Bar chart of Predicted Deaths by Date
    fig2 = px.bar(filtered_df, x="Date", y="Predicted_Deaths", title=f"Predicted Deaths in {selected_month}", text="Predicted_Deaths")
    fig2.update_traces(textposition="outside")

    # KPI Card: Total Deaths
    total_deaths = filtered_df["Deaths"].sum()
    fig3 = go.Figure(go.Indicator(
        mode="number",
        value=total_deaths,
        title={"text": f"Total Deaths in {selected_month}"},
        number={"valueformat": ",", "font": {"size": 40}}
    ))
    fig3.update_layout(height=250, margin=dict(t=50, b=0, l=0, r=0))

    # Graph 4: Line chart for overall trend
    fig4 = px.line(df, x="Date", y=["Deaths", "Predicted_Deaths"], title="Overall Trend of Deaths and Predictions")

    return fig1, fig2, fig3, fig4

if __name__ == "__main__":
    app.run(port=8060, debug=True)
